In [1]:
import pandas as pd
import os
from Bio import Seq
import re

%cd /Users/thiberio/t3ss_effectors/

/Users/thiberio/t3ss_effectors


In [29]:
for filename in os.listdir('.'):
    if filename.startswith('.') or not filename.endswith('.faa'):
        continue
    
    faa       = open(filename).read()
    effectors = {}
    adjacent_genes = []
    for seq_id, genome in re.findall('^>(\S+?)\|(\S+)$', open(filename).read(), re.M):
        if 'contig' in genome  or '[' in genome:
            continue
        
        if genome not in effectors:
            effectors[genome] = []
        effectors[genome].append(seq_id)
    
    for genome in effectors:
        genome_file = open('genomes_4_clustering/%s.faa' % genome).read()
        genome_file = genome_file.split('>')
        genome_file.pop(0)
        
        for position, gene in enumerate(genome_file):
            gene_name = gene.split()[0]
            if gene_name in effectors[genome]:
                adjacent_genes.append(
                    '>downstream_to_%s|%s\n%s' % (gene_name,
                                                  genome,
                                                  '\n'.join(genome_file[position-1].split('\n')[1:]))
                )
                adjacent_genes.append(
                    '>upstream_to_%s|%s\n%s' % (gene_name,
                                                  genome,
                                                  '\n'.join(genome_file[position+1].split('\n')[1:]))
                )


    out = open('adjacent_to_effectors/%s' % filename, 'w')
    out.write(''.join(adjacent_genes))
    out.close()

In [75]:
target_domains = {
    'general chaperone': 'IPR008978 IPR031107 IPR037913 IPR002068 IPR008978 IPR031107 IPR037913'.split(),
    't3ss chaperone'   : 'IPR010261 IPR005415 IPR005416 IPR010261'.split(),
    'histone'          : 'IPR000182'.split(), 
    'transporter'      : 'IPR000412 IPR000620 IPR002178 IPR006143 IPR036709'.split(),
    't3ss'             : 'IPR001712 IPR003537 IPR006135 IPR006302 IPR006307 IPR012842 IPR014773\
                          IPR016379 IPR018035 IPR029025 IPR037168'.split(),
    't4ss'             : 'IPR014128 IPR019476'.split(),
    'related to pathogenesis' : 'IPR003540 IPR035093'.split()
}

In [97]:
%cd /Users/thiberio/t3ss_effectors/adjacent_to_effectors
adjacent_to_histone = {}
adjacent_to_t3ss    = {}
for filename in os.listdir('.'):
    if filename.startswith('.') or not filename.endswith('.gff3'):
        continue
    
    gff = re.split('^##', open(filename).read(), flags=re.M)[4:-1]
    adjacent_to = {}
    for block in gff:
        gene = re.match('sequence-region (?:down|up)stream_to_(\S+)', block).group(1)
        if gene not in adjacent_to:
            adjacent_to[gene] = set()
        adjacent_to[gene].update(re.findall('InterPro:(IPR\d+)', block))
    
    for gene, iprs in adjacent_to.items():
        if iprs.intersection(target_domains['t3ss']):
            if filename not in adjacent_to_t3ss:
                adjacent_to_t3ss[filename] = []
            adjacent_to_t3ss[filename].append(gene)
        if iprs.intersection(target_domains['t3ss chaperone']):
            if filename not in adjacent_to_histone:
                adjacent_to_histone[filename] = []
            adjacent_to_histone[filename].append(gene)


/Users/thiberio/t3ss_effectors/adjacent_to_effectors


In [99]:
print(len(adjacent_to_histone))
for filename, genes in adjacent_to_histone.items():
    print(filename, len(set(genes)))

13
AopS.faa.gff3 2
PteB.faa.gff3 30
PteC.faa.gff3 5
PteD.faa.gff3 4
PteK.faa.gff3 10
AopO.faa.gff3 4
PteA.faa.gff3 3
PteD.1.faa.gff3 1
PteL.faa.gff3 2
AopH.faa.gff3 2
PteF.faa.gff3 2
AexT.faa.gff3 17
AexU.faa.gff3 28


In [88]:
print(len(adjacent_to_t3ss))
for filename, genes in adjacent_to_t3ss.items():
    print(filename, len(genes))

6
AopX.faa.gff3 33
PteC.faa.gff3 5
PteD.faa.gff3 4
PteG.faa.gff3 7
AexT.faa.gff3 5
AexU.faa.gff3 5
